In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    model_name="llama-3.1-70b-versatile",
    groq_api_key= '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',
    max_tokens=None,
    timeout=None
)

responce = llm.invoke("The first person to land on moon was...")
print(responce.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-45041?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for Production Artist II, Energy - RNWY Apparel Digital Design

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsDigitalProduction Artist II, Energy - RNWY Apparel Digital DesignBea

In [4]:
import json
from langchain_core.prompts import PromptTemplate

# Define the prompt template
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The Scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing 
    the following keys: 'role', 'experience', 'skills', and 'description'.
    Only return the valid JSON. Ensure it is syntactically correct and contains no additional text.
    ### VALID JSON (NO PREAMBLE):
    """
)

# Combine the prompt template with the language model
chain_extract = prompt_extract | llm

# Invoke the chain
res = chain_extract.invoke(input={'page_data': page_data})

# Extract content if res is an AIMessage object
if hasattr(res, "content"):
    res = res.content

# Validate and process the result
try:
    # Parse the result to check if it's valid JSON
    result_json = json.loads(res)
    print("Valid JSON:", result_json)
except json.JSONDecodeError as e:
    print(res)


```json
{
  "role": "Production Artist II, Energy - RNWY Apparel Digital Design",
  "experience": "Advanced in time management, Bachelors Degree in related field or equivalent experience",
  "skills": [
    "High proficiency in Adobe Creative Suite",
    "Apparel Pattern Making knowledge",
    "Sewing & Garment Construction knowledge",
    "Comprehensive understanding of textile sciences",
    "Comprehensive understanding of apparel trims",
    "3D design program experience",
    "Tech Pack Creation Knowledge and skills",
    "CAD Line Art creation skills"
  ],
  "description": "Support the Apparel Designers to create and finalize garments details across many seasonal collections, create and maintain accurate BOM’s, assist designers with creating physical mood, trim and swatch boards, and work with product management team to send packages to external partners and factories."
}
```


In [5]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res)
json_res

{'role': 'Production Artist II, Energy - RNWY Apparel Digital Design',
 'experience': 'Advanced in time management, Bachelors Degree in related field or equivalent experience',
 'skills': ['High proficiency in Adobe Creative Suite',
  'Apparel Pattern Making knowledge',
  'Sewing & Garment Construction knowledge',
  'Comprehensive understanding of textile sciences',
  'Comprehensive understanding of apparel trims',
  '3D design program experience',
  'Tech Pack Creation Knowledge and skills',
  'CAD Line Art creation skills'],
 'description': 'Support the Apparel Designers to create and finalize garments details across many seasonal collections, create and maintain accurate BOM’s, assist designers with creating physical mood, trim and swatch boards, and work with product management team to send packages to external partners and factories.'}

In [6]:
type(json_res)

dict

In [7]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [8]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [9]:
links = collection.query(query_texts=['Experience in Java', 'Experience in Python'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [10]:
job = json_res
job['skills']

['High proficiency in Adobe Creative Suite',
 'Apparel Pattern Making knowledge',
 'Sewing & Garment Construction knowledge',
 'Comprehensive understanding of textile sciences',
 'Comprehensive understanding of apparel trims',
 '3D design program experience',
 'Tech Pack Creation Knowledge and skills',
 'CAD Line Art creation skills']

In [11]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}]]

In [12]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Abhijit Pandit, a Automation tester at Cognizant. Cognizant is an Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Cognizant 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Cognizant's portfolio: {link_list}
        Remember you are Abhijit, Tester at Cognizant. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})

if hasattr(res, "content"):
    res = res.content

# Validate and process the result
try:
    # Parse the result to check if it's valid JSON
    result_json = json.loads(res)
    print("Valid JSON:", result_json)
except json.JSONDecodeError as e:
    print(res)

Subject: Empowering Apparel Design with Cognizant's Expertise

Dear Hiring Manager,

I came across the job description for Production Artist II, Energy - RNWY Apparel Digital Design, and I am excited to introduce Cognizant as a potential partner to support your apparel design needs. With our expertise in automation and digital transformation, we can help streamline your design processes, enhance efficiency, and reduce costs.

Our team at Cognizant has extensive experience in providing tailored solutions to various industries, including fashion and apparel. We can leverage our capabilities in areas such as 3D design, CAD line art creation, and tech pack creation to support your designers in creating and finalizing garment details. Our expertise in textile sciences and apparel trims can also help you make informed decisions about fabric selection, color palette, and other design elements.

To demonstrate our capabilities, I would like to highlight some of our relevant portfolios:

* http